In [1]:
import os
import numpy as np
import dill
import pickle
from bottleneck import push
from fancyimpute import IterativeImputer
from miss_utils import *
%load_ext autoreload
%autoreload 2

In [2]:
data_path = 'extracts/'
with open(os.path.join(data_path, '3d_tensor_set_a.pkl'), 'rb') as f:
    origional_a = pickle.load(f)
    
with open(os.path.join(data_path, '3d_tensor_set_b.pkl'), 'rb') as f:
    origional_b = pickle.load(f)    

with open(os.path.join(data_path, '3d_tensor_set_c.pkl'), 'rb') as f:
    origional_c = pickle.load(f)
    
origional = np.vstack([origional_a,origional_b,origional_c]) 
mask=~np.isnan(origional)*1

In [3]:

mins = []
maxs = []
flatten= origional.reshape(origional.shape[0]*origional.shape[1], origional.shape[2])

for i in range(origional.shape[2]):

        mins.append(np.nanmin(flatten[:,i]))

        maxs.append(np.nanmax(flatten[:,i]))

normalized =normalize(origional,mins,maxs)

Patient level introduce

In [4]:
seed = 1234
new_tensor,miss_indices,new_mask=introduce_miss_cols_last(normalized,mask, 0.5,seed, [1,2,20,27,31])
LO =push(new_tensor, axis=1)
LOCV= np.where(np.isnan(LO), 0, LO)
zero= np.where(np.isnan(new_tensor), 0, new_tensor)
means, stds = [],[]
flatten= new_tensor.reshape(origional.shape[0]*origional.shape[1], 35)
for i in range(35):
    means.append(np.nanmean(flatten[:,i]))
    stds.append(np.nanstd(flatten[:,i]))
mean_imputed =mean_fill(new_tensor, means)
mean_imputed =mean_fill(new_tensor, means)
mask_miss=~np.isnan(new_tensor)*1
imputed=prepare_fills(new_tensor, means)
IMM= create_individualized_missingness_mask(mask_miss)

In [5]:
with open('extracts/X_0.5_'+str(seed)+'col_last.pkl', 'wb') as outfile:
    dill.dump(new_tensor, outfile, pickle.HIGHEST_PROTOCOL)  
with open('extracts/new_miss_oo_person_0.5_'+str(seed)+'col_last.pkl', 'wb') as outfile:
    dill.dump(zero, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/new_miss_0.5_person_back_forward_'+str(seed)+'col_last.pkl', 'wb') as outfile:
    dill.dump(imputed, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/mask_miss_person_0.5_'+str(seed)+'col_last.pkl', 'wb') as outfile:
    dill.dump(mask_miss, outfile, pickle.HIGHEST_PROTOCOL)  
with open('extracts/IMM_person_0.5_'+str(seed)+'col_last.pkl', 'wb') as outfile:
    dill.dump(IMM, outfile, pickle.HIGHEST_PROTOCOL)  

In [6]:
mice_impute = IterativeImputer()
flatten= new_tensor.reshape(new_tensor.shape[0]*new_tensor.shape[1], new_tensor.shape[2])
MICE_ = mice_impute.fit_transform(flatten)
MICE=MICE_.reshape(new_tensor.shape[0], new_tensor.shape[1], new_tensor.shape[2])

In [28]:
experiment_name = "IMM_1234_05_most_col_last"
saved_path = 'data/'
observed_only_real = np.load(saved_path + experiment_name + '.npz')["observed_only_real"]
observed_only_rec = np.load(saved_path + experiment_name + '.npz')["observed_only_rec"]
IMM_real = np.load(saved_path + experiment_name + '.npz')["IMM_real"]
IMM_rec = np.load(saved_path + experiment_name + '.npz')["IMM_rec"]
with open(os.path.join('SAITS_0.5_1234_col_last.pkl'), 'rb') as f:
           SAITS = pickle.load(f)    
with open(os.path.join('BRITS_0.5_1234_col_last.pkl'), 'rb') as f:
            BRITS = pickle.load(f)  
experiment_name = "miss_final_loss_no_hidden_01_temp_oo_new_weight_01_IMMRec_001ct"
saved_path = 'data/'
observed_only_real2 = np.load(saved_path + experiment_name + '.npz')["observed_only_real"]
observed_only_rec2 = np.load(saved_path + experiment_name + '.npz')["observed_only_rec"]
IMM_real2 = np.load(saved_path + experiment_name + '.npz')["IMM_real"]
IMM_rec2 = np.load(saved_path + experiment_name + '.npz')["IMM_rec"]       

In [29]:
imputed_ours2 =(mask_miss * (observed_only_real)+ ((1-mask_miss) *observed_only_rec))
imputed_ours =(mask_miss * (observed_only_real)+ ((1-mask_miss) *IMM_rec))

imputed_ours_oo =(mask_miss * (observed_only_real)+ ((1-mask_miss) *observed_only_rec2))
imputed_our_imm =(mask_miss * (observed_only_real)+ ((1-mask_miss) *IMM_rec2))

In [30]:
flat= np.zeros((12000,48,35)).reshape(-1)
flat[miss_indices] = 1
indicating_mask=flat.reshape(12000,48,35)

In [31]:
indicating_mask=indicating_mask.astype(bool)

In [32]:
MICE.shape

(12000, 48, 35)

In [33]:
MICE =MICE.copy()

Calculate Patient level (mean,std)

In [34]:
print("MSE")
print("----------------------")
get_patient_level_MSE_array(LOCV, normalized, indicating_mask, "LOCF") 
get_patient_level_MSE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(imputed_ours, normalized, indicating_mask, "IGNITE OLD ") # cl
get_patient_level_MSE_array(imputed_ours2, normalized, indicating_mask, "IGNITE OLD top") # cl
get_patient_level_MSE_array(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_patient_level_MSE_array(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

MSE
----------------------
LOCF 0.0054349707835787305 0.00521291338070695
mean 0.0030327951984902616 0.0027896234208345033
mice 0.0023318222681979134 0.002679641443907481
BRITS 0.0009945094867971906 0.0022372581030061715
SAITS 0.0009647949821104391 0.001894751215964682
IGNITE OLD  0.016535653246051236 0.01447299394277894
IGNITE OLD top 0.002924663368372656 0.0027376165749880446
IGNITE new OO 0.002965006258819973 0.0027586590931987327
IGNITE new IMM 0.016959763441288616 0.014597505036093306


In [27]:
print("RMSE")
print("----------------------")
get_patient_level_RMSE_array(LOCV, normalized, indicating_mask, "LOCF") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(imputed_ours, normalized, indicating_mask, "IGNITE OLD ") # cl
get_patient_level_RMSE_array(imputed_ours2, normalized, indicating_mask, "IGNITE OLD top") # cl
get_patient_level_RMSE_array(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_patient_level_RMSE_array(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

RMSE
----------------------
LOCF 0.0663738886167413 0.0320854748051381
mean 0.0509218378074626 0.02097049433849221
mice 0.04432962995444396 0.019149573788990144
BRITS 0.027587212211526577 0.015279241119682267
SAITS 0.027468121136260615 0.014501631058406914
IGNITE OLD  0.11741993898907725 0.05242338384588129
IGNITE OLD top 0.04991151669589946 0.02082075573766728
IGNITE new OO 0.050138848509108866 0.020845316765847898
IGNITE new IMM 0.08808640644451138 0.05130578329688935


In [35]:
print("MSE")
print("----------------------")
get_patient_level_MAE_array(LOCV, normalized, indicating_mask, "LOCF") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(imputed_ours, normalized, indicating_mask, "IGNITE") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(imputed_ours2, normalized, indicating_mask, "IGNITE top") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_patient_level_MAE_array(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

MSE
----------------------
LOCF 0.03325223735738138 0.01563343774009006
mean 0.037742596723239534 0.01581863533118696
mice 0.031599431905843016 0.013357767975053931
BRITS 0.01782030039404645 0.007340252355534271
SAITS 0.0182125276999495 0.007070615569330256
IGNITE 0.09389948160569164 0.044121105180653375
IGNITE top 0.03691510986401683 0.015549909488774456
IGNITE new OO 0.037230751283740114 0.015626881586192333
IGNITE new IMM 0.09580527022988805 0.04425275936578016


Calculate Population level sum

In [36]:
print("RMSE")
print("----------------------")
get_population_RMSE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_RMSE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl
get_population_RMSE(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_population_RMSE(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

RMSE
----------------------
LOCF 0.07553743732352447
mean 0.05593420121496634
mice 0.048004549554692216
BRITS 0.032319640357327976
SAITS 0.03190011989423806
IGNITE 0.1366368500489498
IGNITE new OO 0.05533381050776883
IGNITE new IMM 0.13841311183961122


In [39]:
get_population_MSE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MSE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl
get_population_MSE(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_population_MSE(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

LOCF 0.0057059044374053874
mean 0.003128634865556342
mice 0.0023044367779489006
BRITS 0.0010445591528270234
SAITS 0.001017617649266763
IGNITE 0.018669628791299193
IGNITE new OO 0.003061830585309668
IGNITE new IMM 0.019158189529124722


In [38]:
get_population_MAE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MAE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl
get_population_MAE(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_population_MAE(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

LOCF 0.035177585439238095
mean 0.03875447804775543
mice 0.031465663581384036
BRITS 0.018518808848733366
SAITS 0.018958105328920588
IGNITE 0.1025071435677299
IGNITE new OO 0.038222598505533646
IGNITE new IMM 0.10460567637835318


get weighted population sum

In [130]:
get_population_MAE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MAE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

LOCF 0.03517758543880703
mean 0.03875447804726615
mice 0.03146566358097437
BRITS 0.018518808848502315
SAITS 0.01895810532868447
IGNITE 0.10250714356651262


In [ ]:
get_population_RMSE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_RMSE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [ ]:
get_population_MSE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MSE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [113]:
# imputed_ours =(new_mask * (normalized)+ ((1-new_mask) *IMM_rec))
imputed_ours[1,:,1]

array([0.163161188, 0.293333322, 0.201798677, 0.293333322, 0.293333322,
       0.198350668, 0.293333322, 0.293333322, 0.293333322, 0.195991457,
       0.266666681, 0.196183771, 0.266666681, 0.195206344, 0.266666681,
       0.195638478, 0.266666681, 0.266666681, 0.189469635, 0.192066073,
       0.266666681, 0.191787601, 0.187004209, 0.190153092, 0.187479317,
       0.191189557, 0.185786545, 0.189145386, 0.186790168, 0.266666681,
       0.185274869, 0.188726217, 0.226666674, 0.190633208, 0.230000004,
       0.188388258, 0.186282694, 0.270000011, 0.185123295, 0.233333334,
       0.186240375, 0.223333329, 0.25       , 0.188101232, 0.239999995,
       0.263333321, 0.185175657, 0.188034177])

In [151]:
new_tensor[1,:,2]

array([        nan,         nan, 0.238095238,         nan, 0.212454212,
               nan,         nan,         nan,         nan,         nan,
               nan, 0.227106227,         nan, 0.241758242, 0.223443223,
       0.216117216,         nan,         nan, 0.230769231, 0.230769231,
               nan, 0.21978022 , 0.230769231, 0.216117216,         nan,
       0.19047619 , 0.241758242,         nan,         nan,         nan,
               nan,         nan,         nan,         nan, 0.21978022 ,
               nan,         nan, 0.212454212,         nan,         nan,
       0.212454212,         nan,         nan,         nan,         nan,
               nan,         nan,         nan])

In [154]:
(observed_only_rec[1,:,2])*indicating_mask[1,:,2]

array([0.         , 0.211850792, 0.         , 0.216299742, 0.         ,
       0.206679374, 0.201776654, 0.20237413 , 0.200268894, 0.198209465,
       0.197073847, 0.         , 0.197919309, 0.         , 0.         ,
       0.         , 0.198786408, 0.198846161, 0.         , 0.         ,
       0.198718756, 0.         , 0.         , 0.         , 0.198276073,
       0.         , 0.         , 0.197994381, 0.197918147, 0.197775275,
       0.197767854, 0.197647661, 0.197618574, 0.197545499, 0.         ,
       0.197447658, 0.197365165, 0.         , 0.19729659 , 0.19728449 ,
       0.         , 0.197126418, 0.197099   , 0.197112113, 0.197136104,
       0.197057396, 0.         , 0.         ])

In [147]:
(normalized[1,:,2])*indicating_mask[1,:,2]

array([        nan, 0.201465201, 0.         , 0.208791209, 0.         ,
       0.205128205, 0.205128205, 0.201465201, 0.26007326 , 0.234432234,
       0.216117216, 0.         , 0.230769231, 0.         , 0.         ,
       0.         , 0.21978022 , 0.227106227, 0.         , 0.         ,
       0.205128205, 0.         , 0.         , 0.         , 0.212454212,
       0.         , 0.         , 0.227106227, 0.230769231, 0.230769231,
       0.21978022 , 0.186813187, 0.205128205, 0.172161172, 0.         ,
       0.223443223, 0.201465201, 0.         , 0.278388278, 0.197802198,
       0.         , 0.205128205, 0.227106227, 0.21978022 , 0.208791209,
       0.19047619 ,         nan,         nan])

In [155]:
(IMM_rec[1,:,2])*indicating_mask[1,:,2]

array([0.         , 0.123193696, 0.         , 0.144369543, 0.         ,
       0.140029833, 0.136705995, 0.137488201, 0.137239143, 0.137370288,
       0.135880515, 0.         , 0.12810199 , 0.         , 0.         ,
       0.         , 0.123576209, 0.123911545, 0.         , 0.         ,
       0.120555371, 0.         , 0.         , 0.         , 0.116876215,
       0.         , 0.         , 0.12140888 , 0.113718882, 0.113832183,
       0.110035978, 0.118242338, 0.111177966, 0.111466482, 0.         ,
       0.1156196  , 0.109348707, 0.         , 0.105569258, 0.113731131,
       0.         , 0.108384311, 0.104130581, 0.112276919, 0.106889881,
       0.107466564, 0.         , 0.         ])

In [157]:
(SAITS[1,:,2])*indicating_mask[1,:,2]

array([0.         , 0.192548066, 0.         , 0.208770037, 0.         ,
       0.200065434, 0.190329626, 0.183368295, 0.232846528, 0.20054622 ,
       0.200628713, 0.         , 0.216882274, 0.         , 0.         ,
       0.         , 0.204393581, 0.215899035, 0.         , 0.         ,
       0.199002475, 0.         , 0.         , 0.         , 0.240798563,
       0.         , 0.         , 0.21760723 , 0.236119896, 0.236788899,
       0.233477294, 0.224486515, 0.215674311, 0.213041961, 0.         ,
       0.21756433 , 0.195513174, 0.         , 0.192096353, 0.193308443,
       0.         , 0.203336   , 0.226075724, 0.223535985, 0.211680725,
       0.189130098, 0.         , 0.         ])